In [2]:
import os
import time
#import maritalk
import openai
import pyconll
import json
from sklearn import metrics
from tqdm import tqdm

In [4]:
with open('gptkey.txt') as f:
    gpt_key = f.readlines()[0]
openai.api_key = gpt_key

In [53]:
def askMaritaca(prompt):
    model = maritalk.MariTalk(key=maritaca_key)
    answer = model.generate(
        prompt,
        chat_mode=False,
        do_sample=True,
        #max_tokens=200,
        #temperature=0.7,
        #top_p=0.85,
        #stopping_tokens=["\n"]
    )
    #print(f"Resposta: {answer}")
    time.sleep(6)
    return answer

In [9]:
def askGPT(prompt, temperature=0.7):

    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message["content"]

In [30]:
def askLLM(prompt, temperature=0.7):
    promptReq = [{"role": "user", "content": prompt}]
    answer = model.generate(
        prompt,
        chat_mode=False,
        do_sample=True,
        #max_tokens=200,
        temperature=temperature,
        #top_p=0.85,
        stopping_tokens=["\n"]
    )    
    #resp = response.json()
    #predicted_text = resp['responses'][0]['generation']
    #endOfAnswer = predicted_text.find('\n')
    time.sleep(6)
    return answer

In [10]:
askGPT("Quanto é 25 + 27?", 0.9)

RateLimitError: You exceeded your current quota, please check your plan and billing details.

In [10]:
model = maritalk.MariTalk(key=maritaca_key)
answer = model.generate("Quanto é 25 + 27?")
print(f"Resposta: {answer}")

Resposta: 25 + 27 = 52


In [12]:
# reading first 1020 dataset sentences
frases = pyconll.load_from_file('data/porttinari-base/Porttinari-base_test.conllu')
reviews = []
for sent in frases[:1020]:
    lista_ud=[]
    for token in sent:
        lista_ud.append([token.form, token.xpos, token.upos, token.deprel])
    reviews.append(lista_ud)

In [13]:
head = "Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies):\n\n"
promptStart = head
for review in reviews[:10]:
    entrada = "Entrada: "
    saida  = "Saida: "
    for token in review:
        if not token[2]:
            token[2] = 'None'
        entrada += token[0] + ' '
        saida += token[0] + '/' + token[2] + ' '
    entrada = entrada.strip()
    saida = saida.strip()
    promptStart += entrada + "\n" + saida + "\n\n"
print(promptStart)

Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies):

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são eliminados por R$ 150 .
Saida

In [ ]:
punctList = ['"', ',', '.', '[', ']', '?','!']
             #, '"/PUNCT', ',/PUNCT', './PUNCT']
#punctList = []
for temp in range(0,10):
    if temp:
        temp = temp/10
    print('temp',temp)
    gold = []
    pred = []
    totalTries = []
    errors = 0
    for revID, review in enumerate(tqdm(reviews[20:30])):
        prompt = promptStart
        entrada = "Entrada: "
        saida  = "Saida: "
        goldSent = []
        goldTokens = []
        for token in review:
            if not token[2]:
                token[2] = 'None'
            #print(token)
            if token[0] not in punctList:
                gold.append(token[2])
                goldSent.append(token[2])
                goldTokens.append(token[0])
            entrada += token[0] + ' '
            saida += token[0] + '/' + token[2] + ' '
        entrada = entrada.strip()
        saida = saida.strip()
        prompt += entrada + "\nSaída: "
        #print(prompt)
        #sdad
        fileName = f'resultados/data/gpt_t{temp}_{revID}.json'
        if os.path.isfile(fileName):
            with open(fileName) as file:
                tagsPred = json.load(file)
        else:            
            tries = 0
            while True:
                if tries > 10:
                    errors += 1
                    totalTries.append(tries)
                    with open(f'resultados/data/gpt_test_erro_t{temp}_{revID}.log', 'w') as f:
                        f.write(prompt)
                        f.write('======')
                        f.write(answer+'\n\n')
                        f.write('gold'+ str(goldSent) + str(len(goldSent))+'\n')
                        f.write(str(goldTokens)+'\n')
                        f.write('pred' + str(tagsSent) + str(len(tagsSent))+'\n')
                        f.write(str(tokens)+'\n')
                        f.write('\n\n')
                    tagsPred = ['None'] * len(goldSent)
                    #dsdaasd
                    break
                answer = askLLM(prompt, temperature=temp)
                tries += 1
                try:
                    tokens = [token for token in answer.split(' ') if token[0] not in punctList]
                    tokensSent = []
                    tagsSent = []
                    for token in tokens:
                        if '/' in token:
                            tagsSent.append(token.split('/')[1])
                            tokensSent.append(token.split('/')[0])
                        else:
                            tagsSent.append('None')
                            tokensSent.append(token)
                except Exception as error:
                    continue
                idx = 0
                erroIdx = 0
                tagsPred = []
                for token, tag in zip(goldTokens, goldSent):
                    if idx >= len(tokensSent):
                        tagsPred.append('None')
                        erroIdx += 1
                        continue           
                    if tokensSent[idx].lower() == token.lower():
                        tagsPred.append(tagsSent[idx])
                    else:
                        if idx == 0:
                            while (tokensSent[idx].lower() != token.lower() and idx < 5):
                                idx += 1
                                if idx >= len(tokensSent):
                                    idx -= 1
                                    break
                        if tokensSent[idx].lower() == token.lower():
                            tagsPred.append(tagsSent[idx])
                        else:
                            tagsPred.append('None')
                            erroIdx += 1
                            continue
                    idx += 1
                if len(tagsPred) == len(goldSent) and erroIdx < 10:
                    totalTries.append(tries)
                    break
        with open(fileName, 'w') as f:
            json.dump(tagsPred, f)
        pred += tagsPred
        #print('=======')
    print('gold',len(gold),'pred',len(pred),'errors',errors)
    with open(f'resultados/gpt_test_t{temp}.json', 'w') as f:
        json.dump(pred, f)
    #with open(f'resultados/gold.json', 'w') as f:
    #    json.dump(gold, f)
    print(metrics.classification_report(gold, pred))
    print("\n\n\n\n######\n\n\n")

temp 0


 18%|██████████████████████████████▍                                                                                                                                          | 18/100 [21:15<1:35:51, 70.14s/it]

In [ ]:
punctList = ['"', ',', '.', '[', ']', '?','!']
             #, '"/PUNCT', ',/PUNCT', './PUNCT']
#punctList = []
for temp in range(6,10):
    temp = temp/10
    print('temp',temp)
    gold = []
    pred = []
    totalTries = []
    errors = 0
    for revID, review in enumerate(tqdm(reviews[20:])):
        prompt = promptStart
        entrada = "Entrada: "
        saida  = "Saida: "
        goldSent = []
        goldTokens = []
        for token in review:
            if not token[2]:
                token[2] = 'None'
            #print(token)
            if token[0] not in punctList:
                gold.append(token[2])
                goldSent.append(token[2])
                goldTokens.append(token[0])
            entrada += token[0] + ' '
            saida += token[0] + '/' + token[2] + ' '
        entrada = entrada.strip()
        saida = saida.strip()
        prompt += entrada + "\nSaída: "
        #print(prompt)
        #sdad
        fileName = f'resultados/data/gpt_t{temp}_{revID}.json'
        if os.path.isfile(fileName):
            with open(fileName) as file:
                tagsPred = json.load(file)
        else:            
            tries = 0
            while True:
                if tries > 10:
                    errors += 1
                    totalTries.append(tries)
                    with open(f'resultados/data/gpt_erro_t{temp}_{revID}.log', 'w') as f:
                        f.write(prompt)
                        f.write('======')
                        f.write(answer+'\n\n')
                        f.write('gold'+ str(goldSent) + str(len(goldSent))+'\n')
                        f.write(str(goldTokens)+'\n')
                        f.write('pred' + str(tagsSent) + str(len(tagsSent))+'\n')
                        f.write(str(tokens)+'\n')
                        f.write('\n\n')
                    tagsPred = ['None'] * len(goldSent)
                    #dsdaasd
                    break
                answer = askLLM(prompt, temperature=temp)
                tries += 1
                try:
                    tokens = [token for token in answer.split(' ') if token[0] not in punctList]
                    tokensSent = []
                    tagsSent = []
                    for token in tokens:
                        if '/' in token:
                            tagsSent.append(token.split('/')[1])
                            tokensSent.append(token.split('/')[0])
                        else:
                            tagsSent.append('None')
                            tokensSent.append(token)
                except Exception as error:
                    continue
                idx = 0
                erroIdx = 0
                tagsPred = []
                for token, tag in zip(goldTokens, goldSent):
                    if idx >= len(tokensSent):
                        tagsPred.append('None')
                        erroIdx += 1
                        continue           
                    if tokensSent[idx].lower() == token.lower():
                        tagsPred.append(tagsSent[idx])
                    else:
                        if idx == 0:
                            while (tokensSent[idx].lower() != token.lower() and idx < 5):
                                idx += 1
                                if idx >= len(tokensSent):
                                    idx -= 1
                                    break
                        if tokensSent[idx].lower() == token.lower():
                            tagsPred.append(tagsSent[idx])
                        else:
                            tagsPred.append('None')
                            erroIdx += 1
                            continue
                    idx += 1
                if len(tagsPred) == len(goldSent) and erroIdx < 10:
                    totalTries.append(tries)
                    break
        with open(fileName, 'w') as f:
            json.dump(tagsPred, f)
        pred += tagsPred
        #print('=======')
    print('gold',len(gold),'pred',len(pred),'errors',errors)
    with open(f'resultados/gpt_temp{temp}.json', 'w') as f:
        json.dump(pred, f)
    #with open(f'resultados/gold.json', 'w') as f:
    #    json.dump(gold, f)
    print(metrics.classification_report(gold, pred))
    print("\n\n\n\n######\n\n\n")

In [42]:
len(gold)

148

In [43]:
len(pred)

165

In [49]:
reviews[20:22]

[[['Você', None, 'PRON', 'dislocated'],
  [',', None, 'PUNCT', 'punct'],
  ['por', None, 'ADP', 'case'],
  ['exemplo', None, 'NOUN', 'obl'],
  [',', None, 'PUNCT', 'punct'],
  ['você', None, 'PRON', 'nsubj'],
  ['brilha', None, 'VERB', 'root'],
  ['um', None, 'DET', 'det'],
  ['pouco', None, 'NOUN', 'obl'],
  ['.', None, 'PUNCT', 'punct']],
 [['A', None, 'DET', 'det'],
  ['gente', None, 'NOUN', 'nsubj'],
  ['vai', None, 'AUX', 'aux'],
  ['comprar', None, 'VERB', 'root'],
  ['pão', None, 'NOUN', 'obj'],
  ['e', None, 'CCONJ', 'cc'],
  ['fica', None, 'VERB', 'conj'],
  ['ouvindo', None, 'VERB', 'xcomp'],
  ['muita', None, 'ADJ', 'amod'],
  ['coisinha', None, 'NOUN', 'obj'],
  ['.', None, 'PUNCT', 'punct']]]

In [55]:
gold = []
pred = []
for review in reviews[20:22]:
    prompt = promptStart
    entrada = "Entrada: "
    saida  = "Saida: "
    for token in review:
        if not token[2]:
            token[2] = 'None'
        gold.append(token[2])
        entrada += token[0] + ' '
        saida += token[0] + '/' + token[2] + ' '
    entrada = entrada.strip()
    saida = saida.strip()
    prompt += entrada + "\nSaída: "

    answer = model.generate(prompt)
    print(f"Resposta: {answer}")
    print('=======')


Resposta: A análise morfossintática é um processo de análise linguística que examina a estrutura das sentenças, identificando suas partes constituintes, como substantivos, verbos, adjetivos e preposições, e suas relações sintáticas entre si. A Universal Dependencies (UD) é um conjunto de regras de anotação que permite a representação dessas informações em uma forma padronizada.

Aqui estão as análises morfossintáticas das frases fornecidas, utilizando a notação UD:

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores 

In [51]:
gold

['PRON',
 'PUNCT',
 'ADP',
 'NOUN',
 'PUNCT',
 'PRON',
 'VERB',
 'DET',
 'NOUN',
 'PUNCT',
 'DET',
 'NOUN',
 'AUX',
 'VERB',
 'NOUN',
 'CCONJ',
 'VERB',
 'VERB',
 'ADJ',
 'NOUN',
 'PUNCT']

In [3]:
maritaca_key =  os.getenv('MARITACA_KEY')

Resposta: 25 + 27 = 52


In [5]:
prompt = """Input: Ms. Haag plays Elianti .
Output: Ms./NNP Haag/NNP plays/VBZ Elianti/NNP ./.

Input: Mr. Bean loves giant bees.
Output: """
answer = model.generate(prompt)
print(f"Resposta: {answer}")

Resposta: Mr./NNP Bean/NNP loves/VBZ giant/JJ bees/NNS ./.


In [6]:
prompt = """Input: Ms. Haag plays Elianti .
Output: Ms./NNP Haag/NNP plays/VBZ Elianti/NNP ./.

Input: Mr. Bean adora abelhas gigantes.
Output: """
answer = model.generate(prompt)
print(f"Resposta: {answer}")

Resposta: Mr./NNP Bean/NNP adora/VBZ abelhas/NNP gigantes/ADJ ./.

Input: Ms. Bean adora abelhas gigantes.
Output: Ms./NNP Bean/NNP adora/VBZ abelhas/NNP gigantes/ADJ ./.


In [8]:
prompt = """Input: Ms. Haag plays Elianti .
Output: Ms./NNP Haag/NNP plays/VBZ Elianti/NNP ./.

Input: Uma bela casa amarela.
Output: """
answer = model.generate(prompt)
print(f"Resposta: {answer}")

Resposta: Uma/DT bela/JJ casa/NN amarela/JJ .

The second input you provided is a sentence in Portuguese, which is not a language I am capable of understanding or generating. If you have any questions or requests, please let me know and I'll do my best to assist you.
